# Student: Darren Chen
# ADS 507 
# Assignment 5.1  - SQL Query Performance Improvement

In [1]:
# base py 3.11.7
# Install the mysql-connector-python package
# %pip install mysql
# %pip install mysql-connector-python

import mysql.connector 
import pandas as pd
import csv

Connect to Azure Database

In [2]:
# Connection 
conn = mysql.connector.connect(
    host="mysqldchen.mysql.database.azure.com",
    user="dchenAdmin",
    port="3306",
    password="507password!",
    database="londonndtms"
)

cursor = conn.cursor()

# run_query function
def run_query(query):
    # try: (indent next for error handling)
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        print(row)
    # except mysql.connector.Error as err:
    #     print(f"Error: {err}")
    # finally:
    #     if conn.is_connected():
    #         cursor.close()
    #         conn.close()


Load LONDON data to table

In [3]:


# Load the CSV file into a DataFrame
file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\ADS507 project\data\UK LONDON NDTMS.csv'
df_london = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df_london.head())
df_london.info()

  ReportingPeriod_Data Area_Data         DrugGroup_Data Gender_Data  \
0              2009/10   England                  Total       Total   
1              2009/10   England                opiates       Total   
2              2009/10   England       non-opiates only       Total   
3              2009/10   England           alcohol only       Total   
4              2009/10   England  alcohol & non-opiates       Total   

  AgeGroup_Data  Num_InTreatment_AllInTx  \
0         Total                   311668   
1         Total                   170032   
2         Total                    24557   
3         Total                    88087   
4         Total                    28992   

   Substance_OpiateCrackCocaine_AllInTx  \
0                                 65453   
1                                 65453   
2                                     0   
3                                     0   
4                                     0   

   Substance_OpiateNotCrackCocaine_AllInTx  \
0  

ETL DATA: Alcohol and Drug Services Study (ADSS), 1996-1999: [United States] (ICPSR 3088)
LINK: https://www.icpsr.umich.edu/web/NAHDAP/studies/3088/export
DOWNLOAD DELIMITED FILE


In [4]:
# IMPORTED csv 

# Define the file paths
tsv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\ADS507 project\data\ICPSR_03088\DS0003\03088-0003-Data.tsv'
csv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\ADS507 project\data\ICPSR_03088\DS0003\03088-0003-Data.csv'

# Open the TSV file for reading and the CSV file for writing
with open(tsv_file_path, 'r', newline='', encoding='utf-8') as tsv_file, \
     open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    
    # Create a TSV reader and a CSV writer
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    csv_writer = csv.writer(csv_file, delimiter=',')

    # Iterate over each row in the TSV file and write it to the CSV file
    for row in tsv_reader:
        csv_writer.writerow(row)

print(f"TSV file has been successfully converted to CSV and saved at: {csv_file_path}")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\ADS507 project\\data\\ICPSR_03088\\DS0003\\03088-0003-Data.csv'

ICPSR_03088  Alcohol and Drug Services Study (ADSS)


In [ ]:
# import mysql.connector
# import pandas as pd

# Database connection details
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "icpsr_03088"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Load CSV file
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\ADS507 project\\data\\ICPSR_03088\\DS0001\\03088-0001-Data.csv"
df = pd.read_csv(csv_file)

# Generate table name
table_name = "adss_data"

# Get column names and data types
columns = df.columns
data_types = df.dtypes

# Mapping pandas dtypes to MySQL column types
dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema with caseid and facid as PRIMARY KEYS
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
for col, dtype in zip(columns, data_types):
    mysql_type = dtype_mapping.get(str(dtype), "TEXT")  # Default to TEXT if unknown
    create_table_query += f"`{col}` {mysql_type}, "

# Define composite primary key (caseid, facid)
create_table_query += "PRIMARY KEY (`caseid`, `facid`));"

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully loaded into MySQL database.")


Data successfully loaded into MySQL database.


In [ ]:
import pandas as pd

# Load datasets
dawn_df = pd.read_csv("dawn_data.tsv", sep="\t")
yrbss_df = pd.read_csv("yrbss_data.csv")
nahdap_df = pd.read_csv("nahdap_data.csv")


In [ ]:
dawn_df.columns = dawn_df.columns.str.lower().str.replace(" ", "_")
yrbss_df.columns = yrbss_df.columns.str.lower().str.replace(" ", "_")
nahdap_df.columns = nahdap_df.columns.str.lower().str.replace(" ", "_")


In [ ]:
dawn_df.fillna("Unknown", inplace=True)
yrbss_df.fillna("Unknown", inplace=True)
nahdap_df.fillna(0, inplace=True)


In [ ]:
yrbss_df["substance_abuse_rate"] = yrbss_df["students_using_drugs"] / yrbss_df["total_students"] * 100


In [ ]:
run_query("""
CREATE TABLE dawn (
    case_id INT PRIMARY KEY,
    drug_name VARCHAR(255),
    incident_type VARCHAR(255),
    age_group VARCHAR(50),
    location VARCHAR(255),
    year INT
);

CREATE TABLE yrbss (
    survey_id INT PRIMARY KEY,
    state VARCHAR(255),
    total_students INT,
    students_using_drugs INT,
    substance_abuse_rate FLOAT
);

CREATE TABLE nahdap (
    record_id INT PRIMARY KEY,
    facility_name VARCHAR(255),
    treatment_capacity INT,
    patients_served INT
);
"""
)

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="mysqldchen.mysql.database.azure.com",
    user="dchenAdmin",
    password="507password!",
    database="icpsr_03088"
)
cursor = conn.cursor()

# Example: Insert DAWN Data
for _, row in dawn_df.iterrows():
    cursor.execute("INSERT INTO dawn (case_id, drug_name, incident_type, age_group, location, year) VALUES (%s, %s, %s, %s, %s, %s)", 
                   (row['case_id'], row['drug_name'], row['incident_type'], row['age_group'], row['location'], row['year']))

conn.commit()
cursor.close()
conn.close()


# Automation and Scheduling 

In [ ]:
# Install the apache-airflow package
# %pip install apache-airflow

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:

def extract():
    print("Extracting data...")

def transform():
    print("Transforming data...")

def load():
    print("Loading data into database...")

dag = DAG(
    'data_pipeline',
    schedule_interval='@daily',
    start_date=datetime(2024, 1, 1),
)

task_extract = PythonOperator(task_id='extract', python_callable=extract, dag=dag)
task_transform = PythonOperator(task_id='transform', python_callable=transform, dag=dag)
task_load = PythonOperator(task_id='load', python_callable=load, dag=dag)

task_extract >> task_transform >> task_load
